In [ ]:
gbm=None
params = {
        'task': 'train',
        'application': 'regression',
        'boosting_type': 'gbdt',
        'learning_rate': 0.2,
        'num_leaves': 31,
        'tree_learner': 'serial',
        'min_data_in_leaf': 100,
        'metric': ['l1','l2','rmse'],  # l1:mae, l2:mse
        'max_bin': 255,
        'num_trees': 300
    }
# 第二步，流式读取数据(每次10万)
i=1
for sub_data in pd.read_csv(file, chunksize=100000)
    # 区分特征x和结果Y
    x_data = sub_data[x_cols]
    y_data = sub_data[y_col]
 
    # 创建lgb的数据集
    lgb_train = lgb.Dataset(x_data, y_data.values)
    lgb_eval = lgb.Dataset(test[x_cols], test[y_col].values, reference=lgb_train)
 
    # 第三步：增量训练模型
    # 重点来了，通过 init_model 和 keep_training_booster 两个参数实现增量训练
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=1000,
                    valid_sets=lgb_eval,
                    init_model=gbm,             # 如果gbm不为None，那么就是在上次的基础上接着训练
                    feature_name=x_cols,
                    early_stopping_rounds=10,
                    verbose_eval=False,
                    keep_training_booster=True) # 增量训练 
 
    # 输出模型评估分数
    score_train = dict([(s[1], s[2]) for s in gbm.eval_train()])
    score_valid = dict([(s[1], s[2]) for s in gbm.eval_valid()])
    print('当前模型在训练集的得分是：mae=%.4f, mse=%.4f, rmse=%.4f'%(score_train['l1'], score_train['l2'], score_train['rmse']))
    print('当前模型在测试集的得分是：mae=%.4f, mse=%.4f, rmse=%.4f' % (score_valid['l1'], score_valid['l2'], score_valid['rmse']))
    i += 1